# 🚀 Qwen2-VL vs Qwen3-VL: Complete Comparison & Optimized Inference

This notebook provides:
1. Architecture comparison between Qwen2-VL and Qwen3-VL
2. GPU-optimized configurations for T4, A100, H100, and B200
3. Production-ready inference code

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────────────────┐
│                    QWEN2-VL vs QWEN3-VL KEY DIFFERENCES                             │
├─────────────────────────────────────────────────────────────────────────────────────┤
│  Feature              │ Qwen2-VL                  │ Qwen3-VL                        │
│  ─────────────────────┼───────────────────────────┼─────────────────────────        │
│  Position Encoding    │ 3D RoPE only              │ Learned + RoPE + Interpolate    │
│  MLP Activation       │ QuickGELU                 │ SiLU (configurable)             │
│  Multi-Scale Features │ ❌                        │ ✅ DeepStack                    │
│  Video Token Pruning  │ ❌                        │ ✅ EVS (Efficient Video Samp)   │
│  Max Video Frames     │ 14                        │ 24,576                          │
│  Speculative Decode   │ Basic                     │ ✅ Eagle3 native                │
│  MoE Variants         │ ❌                        │ ✅ Qwen3-VL-30B-A3B             │
└─────────────────────────────────────────────────────────────────────────────────────┘
```


In [ ]:
# Cell 1: GPU Detection & Environment Setup
import subprocess
import sys

print("=" * 80)
print("🔍 GPU DETECTION & ENVIRONMENT SETUP")
print("=" * 80)

# Check GPU
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    cc = torch.cuda.get_device_capability()
    
    print(f"\n✅ GPU: {gpu_name}")
    print(f"✅ Memory: {gpu_memory:.1f} GB")
    print(f"✅ Compute Capability: SM {cc[0]}.{cc[1]}")
    
    # Determine architecture and recommendations
    if cc[0] >= 10:
        arch = "Blackwell (B200)"
        recommended = "Qwen3-VL-8B (128K context) or Qwen2-VL-72B"
    elif cc[0] >= 9:
        arch = "Hopper (H100)"
        recommended = "Qwen3-VL-8B or Qwen3-VL-30B-A3B (MoE)"
    elif cc[0] >= 8:
        arch = "Ampere (A100)"
        recommended = "Qwen3-VL-8B or Qwen2-VL-7B"
    else:
        arch = "Turing (T4)"
        recommended = "Qwen2-VL-2B or Qwen3-VL-4B (4-bit)"
    
    print(f"✅ Architecture: {arch}")
    print(f"✅ Recommended: {recommended}")
    
    GPU_INFO = {"name": gpu_name, "memory": gpu_memory, "sm": cc[0]}
else:
    print("❌ No GPU available!")
    sys.exit(1)

print("\n📦 Installing dependencies...")
%pip install -q vllm>=0.6.0 pillow requests numpy
print("✅ Dependencies installed!")


In [ ]:
# Cell 2: GPU-Optimized Configuration Selection

def get_optimal_config(sm_version: int, vram_gb: float, prefer_qwen3: bool = True):
    """Get optimal vLLM configuration based on GPU."""
    
    # T4 / TURING (SM 7.5, 16GB)
    if sm_version < 8:
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-4B-Instruct",
                "trust_remote_code": True, "dtype": "half",
                "quantization": "bitsandbytes", "load_format": "bitsandbytes",
                "gpu_memory_utilization": 0.92, "max_model_len": 2048,
                "enforce_eager": True, "max_num_seqs": 4,
                "limit_mm_per_prompt": {"image": 1, "video": 1},
                "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 512000},
                "enable_prefix_caching": False, "enable_chunked_prefill": False,
            }
        return {
            "model": "Qwen/Qwen2-VL-2B-Instruct",
            "trust_remote_code": True, "dtype": "half",
            "gpu_memory_utilization": 0.90, "max_model_len": 2048,
            "enforce_eager": True, "max_num_seqs": 4,
            "limit_mm_per_prompt": {"image": 1, "video": 0},
            "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 512000},
            "enable_prefix_caching": False, "enable_chunked_prefill": False,
        }
    
    # A100 / AMPERE (SM 8.0, 40-80GB)
    elif sm_version < 9:
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-8B-Instruct",
                "trust_remote_code": True, "dtype": "bfloat16",
                "gpu_memory_utilization": 0.95, "max_model_len": 16384,
                "enforce_eager": False, "max_num_seqs": 32,
                "limit_mm_per_prompt": {"image": 8, "video": 4},
                "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 1572864},
                "enable_prefix_caching": True, "enable_chunked_prefill": True,
            }
        return {
            "model": "Qwen/Qwen2-VL-7B-Instruct",
            "trust_remote_code": True, "dtype": "bfloat16",
            "gpu_memory_utilization": 0.95, "max_model_len": 16384,
            "enforce_eager": False, "max_num_seqs": 32,
            "limit_mm_per_prompt": {"image": 8, "video": 2},
            "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 1572864},
            "enable_prefix_caching": True, "enable_chunked_prefill": True,
        }
    
    # H100 / HOPPER (SM 9.0, 80GB HBM3)
    elif sm_version < 10:
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-8B-Instruct",
                "trust_remote_code": True, "dtype": "bfloat16",
                "gpu_memory_utilization": 0.95, "max_model_len": 32768,
                "enforce_eager": False, "max_num_seqs": 64,
                "limit_mm_per_prompt": {"image": 16, "video": 8},
                "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 2073600},
                "enable_prefix_caching": True, "enable_chunked_prefill": True,
            }
        return {
            "model": "Qwen/Qwen2-VL-7B-Instruct",
            "trust_remote_code": True, "dtype": "bfloat16",
            "gpu_memory_utilization": 0.95, "max_model_len": 32768,
            "enforce_eager": False, "max_num_seqs": 64,
            "limit_mm_per_prompt": {"image": 16, "video": 4},
            "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 2073600},
            "enable_prefix_caching": True, "enable_chunked_prefill": True,
        }
    
    # B200 / BLACKWELL (SM 10.0, 192GB HBM3e)
    else:
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-8B-Instruct",
                "trust_remote_code": True, "dtype": "bfloat16",
                "gpu_memory_utilization": 0.95, "max_model_len": 131072,  # 128K!
                "enforce_eager": False, "max_num_seqs": 128,
                "limit_mm_per_prompt": {"image": 32, "video": 16},
                "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 4147200},
                "enable_prefix_caching": True, "enable_chunked_prefill": True,
            }
        return {
            "model": "Qwen/Qwen2-VL-72B-Instruct",  # Full 72B on single GPU!
            "trust_remote_code": True, "dtype": "bfloat16",
            "gpu_memory_utilization": 0.95, "max_model_len": 32768,
            "enforce_eager": False, "max_num_seqs": 32,
            "limit_mm_per_prompt": {"image": 16, "video": 8},
            "mm_processor_kwargs": {"min_pixels": 784, "max_pixels": 4147200},
            "enable_prefix_caching": True, "enable_chunked_prefill": True,
        }

# Select configuration - set PREFER_QWEN3 to False for Qwen2-VL
PREFER_QWEN3 = True
config = get_optimal_config(GPU_INFO["sm"], GPU_INFO["memory"], prefer_qwen3=PREFER_QWEN3)

print("⚡ SELECTED CONFIGURATION")
print("=" * 80)
for k, v in config.items():
    print(f"  {k}: {v}")
print("=" * 80)


In [ ]:
# Cell 3: Initialize vLLM Engine
from vllm import LLM, SamplingParams
import time

print("⚡ Initializing vLLM engine...")
print(f"   Model: {config['model']}")

init_start = time.time()
llm = LLM(**config)
init_time = time.time() - init_start

print(f"\n✅ Engine initialized in {init_time:.1f}s")

allocated = torch.cuda.memory_allocated() / (1024**3)
reserved = torch.cuda.memory_reserved() / (1024**3)
print(f"\n📊 GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")


In [ ]:
# Cell 4: Create Test Image
from PIL import Image, ImageDraw

def create_test_image(width=1024, height=1024):
    """Create test dashboard image."""
    img = Image.new('RGB', (width, height), '#0f1420')
    draw = ImageDraw.Draw(img)
    
    # Top bar
    draw.rectangle([0, 0, width, 60], fill='#1a2535')
    draw.text((20, 20), "Dashboard - Analytics", fill='#ffffff')
    
    # Sidebar
    draw.rectangle([0, 60, 200, height], fill='#141d2b')
    items = ["Dashboard", "Analytics", "Reports", "Settings", "Users"]
    for i, item in enumerate(items):
        y = 100 + i * 50
        draw.rectangle([10, y, 190, y + 40], fill='#1f2d42')
        draw.text((30, y + 10), item, fill='#b0b0b0')
    
    # Cards
    for row in range(2):
        for col in range(3):
            x, y = 220 + col * 260, 80 + row * 200
            draw.rectangle([x, y, x + 240, y + 180], fill='#1a2535', outline='#2a3a4f')
            draw.text((x + 20, y + 20), f"Metric {row * 3 + col + 1}", fill='#808080')
            draw.text((x + 20, y + 60), f"${(row * 3 + col + 1) * 1234:,}", fill='#00d084')
    
    # Buttons
    for i, (text, color) in enumerate([("Export", '#3a7bd5'), ("Refresh", '#00a86b')]):
        x = 220 + i * 150
        draw.rectangle([x, height - 80, x + 130, height - 40], fill=color)
        draw.text((x + 20, height - 70), text, fill='#ffffff')
    
    return img

test_image = create_test_image()
print(f"📸 Created test image: {test_image.size}")

# Display thumbnail
thumb = test_image.copy()
thumb.thumbnail((600, 600))
display(thumb)


In [ ]:
# Cell 5: Prompt Format Function
def format_prompt(instruction: str, model_name: str) -> str:
    """Format prompt based on model family."""
    system = "You are a helpful visual assistant. Analyze images carefully."
    
    return (
        f"<|im_start|>system\n{system}<|im_end|>\n"
        f"<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>{instruction}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

print("✅ Prompt functions defined")


In [ ]:
# Cell 6: Run Single Image Inference
sampling_params = SamplingParams(temperature=0.0, max_tokens=512, stop=["<|im_end|>"])

instructions = [
    "Describe this dashboard interface.",
    "What menu items are visible in the sidebar?",
    "What is the total value of all metrics shown?",
]

print("⚡ Running inference...")
print("=" * 80)

model_name = config["model"]
results = []

for i, instruction in enumerate(instructions, 1):
    prompt = format_prompt(instruction, model_name)
    inputs = {"prompt": prompt, "multi_modal_data": {"image": test_image}}
    
    start = time.time()
    outputs = llm.generate([inputs], sampling_params=sampling_params)
    latency = (time.time() - start) * 1000
    
    response = outputs[0].outputs[0].text
    tokens = len(outputs[0].outputs[0].token_ids)
    
    print(f"\n[{i}] {instruction}")
    print(f"    ⏱️  Latency: {latency:.0f}ms | Tokens: {tokens} | Speed: {tokens/(latency/1000):.1f} tok/s")
    print(f"    📝 Response: {response[:200]}..." if len(response) > 200 else f"    📝 Response: {response}")
    
    results.append({"instruction": instruction, "latency": latency, "tokens": tokens})

avg_latency = sum(r["latency"] for r in results) / len(results)
print(f"\n📊 Average latency: {avg_latency:.0f}ms")


In [ ]:
# Cell 7: Batch Inference (High Throughput)
batch_instructions = [
    "Click on Dashboard", "Click on Analytics", "Click on Reports",
    "Click on Settings", "Click on Users", "Click on Export",
    "Click on Refresh", "Click on Metric 1",
]

batch_inputs = [
    {"prompt": format_prompt(inst, model_name), "multi_modal_data": {"image": test_image}}
    for inst in batch_instructions
]

print(f"⚡ Batch inference: {len(batch_inputs)} requests")

batch_start = time.time()
batch_outputs = llm.generate(batch_inputs, sampling_params=sampling_params)
batch_time = (time.time() - batch_start) * 1000

print(f"\n✅ Completed in {batch_time:.0f}ms")
print(f"📈 Throughput: {len(batch_inputs) / (batch_time / 1000):.1f} requests/second")
print(f"📈 Per-request: {batch_time / len(batch_inputs):.0f}ms")


In [ ]:
# Cell 8: Architecture & GPU Summary
print("""
╔═════════════════════════════════════════════════════════════════════════════════════╗
║                    QWEN2-VL vs QWEN3-VL ARCHITECTURE COMPARISON                     ║
╠═════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                     ║
║  ┌──────────────────────┬─────────────────────────┬────────────────────────────┐   ║
║  │ Component            │ Qwen2-VL                │ Qwen3-VL                   │   ║
║  ├──────────────────────┼─────────────────────────┼────────────────────────────┤   ║
║  │ Patch Embedding      │ Conv3D (no bias)        │ Conv3D (WITH bias)         │   ║
║  │ Position Encoding    │ 3D RoPE only            │ Learned + RoPE + Interp    │   ║
║  │ MLP Activation       │ QuickGELU               │ SiLU                       │   ║
║  │ Multi-Scale Features │ ❌                      │ ✅ DeepStack               │   ║
║  │ Video Token Pruning  │ ❌                      │ ✅ EVS                     │   ║
║  │ Max Video Frames     │ 14                      │ 24,576                     │   ║
║  │ Speculative Decode   │ Basic                   │ ✅ Eagle3                  │   ║
║  │ MoE Variants         │ ❌                      │ ✅ Qwen3-VL-30B-A3B        │   ║
║  └──────────────────────┴─────────────────────────┴────────────────────────────┘   ║
╚═════════════════════════════════════════════════════════════════════════════════════╝

╔═════════════════════════════════════════════════════════════════════════════════════╗
║                          GPU PERFORMANCE EXPECTATIONS                               ║
╠═════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                     ║
║  GPU          │ Memory │ Best Model             │ Context │ Latency │ Throughput   ║
║  ─────────────┼────────┼────────────────────────┼─────────┼─────────┼───────────   ║
║  T4           │ 16 GB  │ Qwen2-VL-2B            │ 2K      │ ~800ms  │ ~20 tok/s    ║
║               │        │ Qwen3-VL-4B (4-bit)    │ 2K      │ ~1000ms │ ~18 tok/s    ║
║  A100-80GB    │ 80 GB  │ Qwen3-VL-8B            │ 16K     │ ~250ms  │ ~100 tok/s   ║
║  H100         │ 80 GB  │ Qwen3-VL-8B            │ 32K     │ ~150ms  │ ~150 tok/s   ║
║               │        │ Qwen3-VL-30B-A3B (MoE) │ 16K     │ ~300ms  │ ~60 tok/s    ║
║  B200         │ 192 GB │ Qwen3-VL-8B            │ 128K    │ ~80ms   │ ~200 tok/s   ║
║               │        │ Qwen2-VL-72B           │ 32K     │ ~200ms  │ ~80 tok/s    ║
║                                                                                     ║
╚═════════════════════════════════════════════════════════════════════════════════════╝
""")

print("✅ Notebook complete!")
